![head.png](https://github.com/cafawo/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Homework: Satellite imagery

You will need a Git/GitHub repository to submit your course deliverables. Consult [**slides.ipynb**](https://github.com/cafawo/FinancialDataAnalytics) for help with the tasks below! If you need further assistance, do not hesitate to open a Q&A at https://github.com/cafawo/FinancialDataAnalytics/discussions

In [ ]:
# Packages used in this notebook
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np

### Task:

Set up a google earth engine workflow to display satellite images of a location (or region) of your choosing.

In [2]:
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Define the region of interest (ROI) as Mumbai
mumbai = ee.Geometry.Point([72.8777, 19.0760])  # Coordinates for Mumbai

# Load a Landsat 8 image collection
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
              .filterDate('2023-01-01', '2023-12-31') \
              .filterBounds(mumbai)

# Get the median image
image = collection.median()

# Select the bands to display (e.g., Red, Green, Blue)
bands = ['B4', 'B3', 'B2']

# Create a map centered on Mumbai
Map = geemap.Map(center=[19.0760, 72.8777], zoom=10)

# Add the image to the map
Map.addLayer(image, {'bands': bands, 'min': 0, 'max': 3000}, 'Landsat 8 Image of Mumbai')

# Display the map
Map

/home/codespace/.python/current/lib/python3.10/site-packages/ee/deprecation.py:204: DeprecationWarning: 

Attention required for LANDSAT/LC08/C01/T1_TOA! You are using a deprecated asset.
To ensure continued functionality, please update it by July 1, 2024.
Learn more: https://developers.google.com/earth-engine/landsat_c1_to_c2

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[19.076, 72.8777], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### Task:

Show a time series plot of the average summer temperature in celsius at your location. To do so, choose a relevant data sets from: https://developers.google.com/earth-engine/datasets/

### Task:

Discuss potential data quality issues and solutions. Do the results (temperatures) change if you employ a BAP composition? Add another line to the time series plot, displaying temperatures extracted from the BAP composites.